**Snake Game AI**

# Task
Implement a Snake Game in Python with a Minimax AI agent, including game logic, food generation, collision detection, and a simple graphical user interface to demonstrate the AI's functionality.

**Reasoning**:
The first step in implementing the Snake Game logic is to define the game board dimensions and initialize the game state, including the snake's body, its initial direction, and the score. This will set up the basic parameters for the game.



In [1]:
import random

# 1. Define game board dimensions and initialize game state
BOARD_WIDTH = 20
BOARD_HEIGHT = 20
CELL_SIZE = 20 # This will be used for visualization later, but good to define now

# Snake initial state
snake = [(10, 10), (9, 10), (8, 10)] # List of (x, y) coordinates
direction = 'RIGHT' # Initial direction: UP, DOWN, LEFT, RIGHT
score = 0

# Food initial state (will be generated in the next step)
food = None

print(f"Board dimensions: {BOARD_WIDTH}x{BOARD_HEIGHT}")
print(f"Initial Snake position: {snake}")
print(f"Initial Direction: {direction}")
print(f"Initial Score: {score}")

Board dimensions: 20x20
Initial Snake position: [(10, 10), (9, 10), (8, 10)]
Initial Direction: RIGHT
Initial Score: 0


In [2]:
def generate_food(snake_body):
    while True:
        food_x = random.randint(0, BOARD_WIDTH - 1)
        food_y = random.randint(0, BOARD_HEIGHT - 1)
        new_food = (food_x, food_y)
        if new_food not in snake_body:
            return new_food

# Initialize food after defining the function
food = generate_food(snake)
print(f"Initial Food position: {food}")

Initial Food position: (14, 4)


In [3]:
def move_snake(current_snake, current_direction):
    head_x, head_y = current_snake[0]

    if current_direction == 'UP':
        new_head = (head_x, head_y - 1)
    elif current_direction == 'DOWN':
        new_head = (head_x, head_y + 1)
    elif current_direction == 'LEFT':
        new_head = (head_x - 1, head_y)
    elif current_direction == 'RIGHT':
        new_head = (head_x + 1, head_y)
    else:
        raise ValueError("Invalid direction")

    # Add new head to the front of the snake
    new_snake = [new_head] + current_snake

    # The tail will be removed later in the main game loop if no food is eaten
    # For now, just return the snake with the new head added. The logic for removing the tail
    # will be handled when checking for food collision.
    return new_snake

# Test the move_snake function (optional, just for verification)
print(f"Original snake: {snake}")
next_snake_head_added = move_snake(snake, direction)
print(f"Snake after moving (head added, tail not yet removed): {next_snake_head_added}")

Original snake: [(10, 10), (9, 10), (8, 10)]
Snake after moving (head added, tail not yet removed): [(11, 10), (10, 10), (9, 10), (8, 10)]


In [4]:
def check_collisions(current_snake, current_food, board_width, board_height):
    head = current_snake[0]
    game_over = False
    food_eaten = False

    # Collision with boundaries
    if not (0 <= head[0] < board_width and 0 <= head[1] < board_height):
        game_over = True

    # Collision with self (head collides with any part of the body except the first element itself)
    if head in current_snake[1:]:
        game_over = True

    # Collision with food
    if head == current_food:
        food_eaten = True

    return game_over, food_eaten

# Test collision detection (for boundary and self-collision, need to simulate movement)
# Let's test food collision first with current state
game_over, food_eaten = check_collisions(snake, food, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Initial collision check: Game Over = {game_over}, Food Eaten = {food_eaten}")

# Simulate a boundary collision for testing
snake_boundary_test = [(-1, 10), (0, 10), (1, 10)] # Head outside left boundary
game_over_boundary, _ = check_collisions(snake_boundary_test, food, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Boundary collision test (head at -1,10): Game Over = {game_over_boundary}")

# Simulate a self-collision for testing
snake_self_collision_test = [(10, 10), (10, 11), (10, 10)] # Head collides with tail
game_over_self, _ = check_collisions(snake_self_collision_test, food, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Self-collision test (head at 10,10, also in body): Game Over = {game_over_self}")

Initial collision check: Game Over = False, Food Eaten = False
Boundary collision test (head at -1,10): Game Over = True
Self-collision test (head at 10,10, also in body): Game Over = True


In [5]:
def update_game_state(current_snake, current_direction, current_food, current_score, board_width, board_height):
    new_snake = move_snake(current_snake, current_direction)
    head = new_snake[0]

    game_over, food_eaten = check_collisions(new_snake, current_food, board_width, board_height)

    if food_eaten:
        new_score = current_score + 1
        # Snake grows, so no tail removal
        new_food = generate_food(new_snake) # Generate new food
    else:
        new_score = current_score
        new_snake.pop() # Remove tail if no food eaten
        new_food = current_food # Food remains in place

    # If game is over due to boundary or self collision, food eaten might still be true if it happens on same step
    # But the game state should reflect game_over and no further changes.

    return new_snake, new_food, new_score, game_over

# Test the update_game_state function
print(f"Initial snake: {snake}, food: {food}, score: {score}, direction: {direction}")

# Simulate one step without food eaten
# Make sure food is not directly in front of the snake for this test
if food == (11, 10): # If food is where snake moves, re-generate for a non-eating test
    food = generate_food(snake)
    print(f"Regenerated food for test: {food}")

updated_snake, updated_food, updated_score, is_game_over = update_game_state(snake, direction, food, score, BOARD_WIDTH, BOARD_HEIGHT)
print(f"After 1st update: snake={updated_snake}, food={updated_food}, score={updated_score}, game_over={is_game_over}")

# Simulate eating food (force food to be in front of snake)
original_snake_for_eat_test = [(5,5), (4,5)]
original_score_for_eat_test = 0
original_direction_for_eat_test = 'RIGHT'
food_for_eat_test = (6,5) # Place food directly where snake will move

print(f"\nSimulating food eat: snake={original_snake_for_eat_test}, food={food_for_eat_test}, score={original_score_for_eat_test}")
snake_after_eat, food_after_eat, score_after_eat, game_over_after_eat = update_game_state(
    original_snake_for_eat_test, original_direction_for_eat_test, food_for_eat_test, original_score_for_eat_test,
    BOARD_WIDTH, BOARD_HEIGHT
)
print(f"After eating: snake={snake_after_eat}, food={food_after_eat}, score={score_after_eat}, game_over={game_over_after_eat}")

# Test game over condition due to self-collision (from check_collisions test earlier)
print(f"\nSimulating game over (self-collision): snake={snake_self_collision_test}, food={food}, score={score}")
_, _, _, game_over_self_collision = update_game_state(
    snake_self_collision_test, 'RIGHT', food, score, BOARD_WIDTH, BOARD_HEIGHT
)
print(f"Game over (self-collision): {game_over_self_collision}")

# Test game over condition due to boundary collision (from check_collisions test earlier)
print(f"\nSimulating game over (boundary collision): snake={snake_boundary_test}, food={food}, score={score}")
_, _, _, game_over_boundary_collision = update_game_state(
    snake_boundary_test, 'LEFT', food, score, BOARD_WIDTH, BOARD_HEIGHT
)
print(f"Game over (boundary collision): {game_over_boundary_collision}")

Initial snake: [(10, 10), (9, 10), (8, 10)], food: (14, 4), score: 0, direction: RIGHT
After 1st update: snake=[(11, 10), (10, 10), (9, 10)], food=(14, 4), score=0, game_over=False

Simulating food eat: snake=[(5, 5), (4, 5)], food=(6, 5), score=0
After eating: snake=[(6, 5), (5, 5), (4, 5)], food=(10, 9), score=1, game_over=False

Simulating game over (self-collision): snake=[(10, 10), (10, 11), (10, 10)], food=(14, 4), score=0
Game over (self-collision): False

Simulating game over (boundary collision): snake=[(-1, 10), (0, 10), (1, 10)], food=(14, 4), score=0
Game over (boundary collision): True


In [6]:
import time # For simulating game ticks

def main_game_loop():
    global snake, food, direction, score, BOARD_WIDTH, BOARD_HEIGHT

    game_over = False
    game_tick_speed = 0.2 # seconds per game tick

    # Re-initialize for a clean game start (important if testing functions modified globals)
    snake = [(10, 10), (9, 10), (8, 10)]
    direction = 'RIGHT'
    score = 0
    food = generate_food(snake)

    print("\n--- Starting Game Loop ---")
    print(f"Initial State: Snake={snake}, Food={food}, Score={score}, Direction={direction}")

    while not game_over:
        # In a real game, direction would be updated by player input
        # For now, we'll keep it constant or implement a simple AI later
        # For this basic loop, we'll just let it move forward.

        # Update game state
        snake, food, score, game_over = update_game_state(
            snake, direction, food, score, BOARD_WIDTH, BOARD_HEIGHT
        )

        print(f"Snake: {snake}, Food: {food}, Score: {score}, Game Over: {game_over}")

        if game_over:
            print("Game Over! Final Score:", score)
            break

        time.sleep(game_tick_speed) # Pause for game speed

    print("--- Game Loop Ended ---")

# Run the main game loop
main_game_loop()



--- Starting Game Loop ---
Initial State: Snake=[(10, 10), (9, 10), (8, 10)], Food=(2, 5), Score=0, Direction=RIGHT
Snake: [(11, 10), (10, 10), (9, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(12, 10), (11, 10), (10, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(13, 10), (12, 10), (11, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(14, 10), (13, 10), (12, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(15, 10), (14, 10), (13, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(16, 10), (15, 10), (14, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(17, 10), (16, 10), (15, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(18, 10), (17, 10), (16, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(19, 10), (18, 10), (17, 10)], Food: (2, 5), Score: 0, Game Over: False
Snake: [(20, 10), (19, 10), (18, 10)], Food: (2, 5), Score: 0, Game Over: True
Game Over! Final Score: 0
--- Game Loop Ended ---


In [7]:
def get_next_head_position(head, direction):
    head_x, head_y = head
    if direction == 'UP':
        return (head_x, head_y - 1)
    elif direction == 'DOWN':
        return (head_x, head_y + 1)
    elif direction == 'LEFT':
        return (head_x - 1, head_y)
    elif direction == 'RIGHT':
        return (head_x + 1, head_y)
    return head # Should not happen with valid directions

# Test the helper function
current_head = (10, 10)
print(f"Current head: {current_head}")
print(f"Next head if UP: {get_next_head_position(current_head, 'UP')}")
print(f"Next head if DOWN: {get_next_head_position(current_head, 'DOWN')}")
print(f"Next head if LEFT: {get_next_head_position(current_head, 'LEFT')}")
print(f"Next head if RIGHT: {get_next_head_position(current_head, 'RIGHT')}")

Current head: (10, 10)
Next head if UP: (10, 9)
Next head if DOWN: (10, 11)
Next head if LEFT: (9, 10)
Next head if RIGHT: (11, 10)


In [8]:
def get_possible_moves(current_snake, board_width, board_height):
    possible_directions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    valid_moves = []
    current_head = current_snake[0]

    for direction_to_check in possible_directions:
        # Simulate the next head position
        next_head = get_next_head_position(current_head, direction_to_check)

        # Simulate the snake's body after the move.
        # For collision checking, we need a snake with the new head and the old body (without tail removal yet).
        # The check_collisions function handles this correctly by looking for head in current_snake[1:]
        # so we pass the new_head prepended to the current_snake, and let check_collisions deal with the details.
        temp_snake = [next_head] + current_snake[:-1] # Remove the last segment as if it moved without eating
                                                   # This is important for self-collision check on the next move.

        # Check for collisions with the simulated next state
        game_over, _ = check_collisions(temp_snake, food, board_width, board_height)

        if not game_over:
            valid_moves.append(direction_to_check)

    return valid_moves

# Test the get_possible_moves function
print(f"Current snake: {snake}")
print(f"Current food: {food}")

# Assuming snake is currently [(20, 10), (19, 10), (18, 10)], moving RIGHT
# It should only allow UP and DOWN, as RIGHT is boundary collision, and LEFT would be self collision if snake was longer.
# Let's use a simpler snake for testing initially
snake_test = [(5, 5), (4, 5), (3, 5)]
print(f"\nTesting with snake: {snake_test}")
valid_moves_test = get_possible_moves(snake_test, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Possible moves: {valid_moves_test}")

# Test case where snake is near a wall
snake_near_wall = [(0, 0), (0, 1)] # Head at (0,0), going UP/LEFT would be collision
print(f"\nTesting with snake near wall: {snake_near_wall}")
valid_moves_near_wall = get_possible_moves(snake_near_wall, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Possible moves: {valid_moves_near_wall}")

# Test case to specifically check for self-collision with get_possible_moves
# Snake moves right, then up, then left, resulting in head colliding with body
snake_self_collision_scenario = [(2,2), (2,3), (3,3), (3,2)]
# If moving RIGHT from (2,2), next head is (3,2). (3,2) is also in the body.
# The temp_snake will be [(3,2), (2,2), (2,3), (3,3)], after removing the tail (3,2) is still in the body.
# However, the current check_collisions already accounts for the new head not being in the *rest* of the body.
# For get_possible_moves, we are simulating a *next step* in the game.
# The snake's tail effectively moves forward, so the last segment is effectively gone.
# So, a 'LEFT' move for snake_self_collision_scenario `[(2,2), (2,3), (3,3)]` (with (3,2) gone) should be valid.
# But for snake = [(2,2), (2,3), (3,3), (3,2)], if we move LEFT, the new head is (1,2). This is valid.
# If we consider the current 'snake_self_collision_scenario' as the snake, and want to check if a move `RIGHT` is valid:
# current_head = (2,2)
# next_head_right = (3,2)
# temp_snake = [(3,2), (2,2), (2,3), (3,3)] (original snake, new head prepended, tail removed for check)
# in check_collisions, head (3,2) will be compared with (2,2), (2,3), (3,3). It's not a self-collision with this body definition.
# Let's adjust the `temp_snake` creation for more accurate simulation for `get_possible_moves`.
# The issue is that `check_collisions` is called with `new_snake` from `update_game_state`, which correctly removes the tail if no food is eaten.
# For `get_possible_moves`, we are *predicting* without actual state change. If we move, the tail disappears *unless food is eaten*.
# Since we are just checking for immediate collision, we should assume the tail *would* move.

def get_possible_moves_corrected(current_snake, board_width, board_height):
    possible_directions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    valid_moves = []
    current_head = current_snake[0]

    # Store the global food value to be used in check_collisions, as it's part of its signature.
    # For get_possible_moves, food location doesn't determine collision, only if the space is occupied by snake or wall.
    # The check_collisions food_eaten return is not used here.
    # We also pass a dummy food position for the `check_collisions` function because we only care about `game_over` here.
    dummy_food = (-1,-1) # A coordinate outside the board to ensure it's not the head position

    for direction_to_check in possible_directions:
        next_head = get_next_head_position(current_head, direction_to_check)

        # Create a temporary snake for collision checking.
        # The new head is added, and the tail is removed, mimicking a non-food-eating move.
        # This ensures self-collision detection is accurate for the *next* state.
        temp_snake = [next_head] + current_snake[:-1]

        # It's crucial that `check_collisions` correctly identifies self-collision
        # using `head in current_snake[1:]` for the *simulated* snake.
        game_over, _ = check_collisions(temp_snake, dummy_food, board_width, board_height)

        if not game_over:
            valid_moves.append(direction_to_check)

    return valid_moves

# Re-testing with corrected function
snake = [(10, 10), (9, 10), (8, 10)] # Global snake state
food = (2,5) # Global food state
print(f"\nTesting corrected function with current snake: {snake}, food: {food}")
current_valid_moves = get_possible_moves_corrected(snake, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Current possible moves: {current_valid_moves}")

# Test with a snake configuration that could lead to self-collision
snake_for_self_collision_test = [(2,2), (2,3), (3,3), (3,2)] # A snake that makes a 'U' shape
print(f"\nTesting self-collision scenario with snake: {snake_for_self_collision_test}")
# Head is (2,2). If it moves RIGHT, next_head=(3,2).
# temp_snake = [(3,2), (2,2), (2,3), (3,3)]. Here, (3,2) is in temp_snake[1:] = [(2,2), (2,3), (3,3)] is FALSE.
# If it moves DOWN, next_head=(2,3). temp_snake = [(2,3), (2,2), (2,3), (3,3)]. Here, (2,3) is in temp_snake[1:] = [(2,2), (2,3), (3,3)] is TRUE.
# So, 'DOWN' should be an invalid move.
valid_moves_self_collision = get_possible_moves_corrected(snake_for_self_collision_test, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Possible moves: {valid_moves_self_collision}")

# The current global `snake` state from previous cell is [(20, 10), (19, 10), (18, 10)].
# This snake's head is (20,10) which is out of bounds (BOARD_WIDTH is 20, so valid x is 0-19).
# The last `main_game_loop` run ended with game_over = True because the snake went out of bounds.
# Let's reset the global snake for testing if it's necessary to avoid errors.
# For safety and clear testing of `get_possible_moves_corrected`, we use local test variables.


Current snake: [(20, 10), (19, 10), (18, 10)]
Current food: (2, 5)

Testing with snake: [(5, 5), (4, 5), (3, 5)]
Possible moves: ['UP', 'DOWN', 'RIGHT']

Testing with snake near wall: [(0, 0), (0, 1)]
Possible moves: ['DOWN', 'RIGHT']

Testing corrected function with current snake: [(10, 10), (9, 10), (8, 10)], food: (2, 5)
Current possible moves: ['UP', 'DOWN', 'RIGHT']

Testing self-collision scenario with snake: [(2, 2), (2, 3), (3, 3), (3, 2)]
Possible moves: ['UP', 'LEFT', 'RIGHT']


In [9]:
def evaluate_state(current_snake, current_food, current_score, board_width, board_height):
    # Prioritize higher scores
    score_value = current_score * 100 # Give significant weight to score

    # Calculate Manhattan distance to food
    head_x, head_y = current_snake[0]
    food_x, food_y = current_food
    distance_to_food = abs(head_x - food_x) + abs(head_y - food_y)

    # Reward getting closer to food (negative distance is better)
    food_heuristic = -distance_to_food * 2 # Multiply by a factor to make it more impactful

    # Penalize being too close to walls or self-collision scenarios (though game_over will handle actual death)
    # This heuristic aims to guide the snake away from immediate danger or trapped positions.
    # For this simple heuristic, we'll primarily focus on score and food distance.
    # More complex heuristics could involve pathfinding or free space calculations.

    # Length of the snake is also good, longer snake means better state
    length_heuristic = len(current_snake) * 10 # Reward longer snake

    # Sum up the heuristics
    return score_value + food_heuristic + length_heuristic

# Test the evaluate_state function

# Scenario 1: Initial state (snake far from food, low score)
snake_initial = [(10, 10), (9, 10), (8, 10)]
food_initial = (1, 1) # Far away
score_initial = 0
eval1 = evaluate_state(snake_initial, food_initial, score_initial, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Scenario 1 (Initial): Snake={snake_initial}, Food={food_initial}, Score={score_initial} -> Eval Score: {eval1}")

# Scenario 2: Snake close to food, same length, same score
snake_close_to_food = [(10, 10), (9, 10), (8, 10)]
food_close = (11, 10) # Right next to head
score_close = 0
eval2 = evaluate_state(snake_close_to_food, food_close, score_close, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Scenario 2 (Close to food): Snake={snake_close_to_food}, Food={food_close}, Score={score_close} -> Eval Score: {eval2}")

# Scenario 3: Snake ate food, longer, higher score, new food location
snake_ate_food = [(11, 10), (10, 10), (9, 10), (8, 10)] # Longer snake
food_new_loc = (5, 5) # New food, some distance away
score_increased = 1
eval3 = evaluate_state(snake_ate_food, food_new_loc, score_increased, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Scenario 3 (After eating): Snake={snake_ate_food}, Food={food_new_loc}, Score={score_increased} -> Eval Score: {eval3}")

# Scenario 4: Snake same as scenario 1 but with higher score
snake_high_score = [(10, 10), (9, 10), (8, 10)]
food_high_score = (1, 1)
score_high = 5
eval4 = evaluate_state(snake_high_score, food_high_score, score_high, BOARD_WIDTH, BOARD_HEIGHT)
print(f"Scenario 4 (High score): Snake={snake_high_score}, Food={food_high_score}, Score={score_high} -> Eval Score: {eval4}")

Scenario 1 (Initial): Snake=[(10, 10), (9, 10), (8, 10)], Food=(1, 1), Score=0 -> Eval Score: -6
Scenario 2 (Close to food): Snake=[(10, 10), (9, 10), (8, 10)], Food=(11, 10), Score=0 -> Eval Score: 28
Scenario 3 (After eating): Snake=[(11, 10), (10, 10), (9, 10), (8, 10)], Food=(5, 5), Score=1 -> Eval Score: 118
Scenario 4 (High score): Snake=[(10, 10), (9, 10), (8, 10)], Food=(1, 1), Score=5 -> Eval Score: 494


In [10]:
def minimax(current_snake, current_food, current_score, current_direction, depth, max_depth):
    # Base Case 1: Game Over (collision)
    # We need to simulate the next step to check if it leads to immediate game over
    # For the purpose of minimax, we'll try to get the current head and check if it's already a losing state.
    # However, the recursion happens *after* a move is made, so the 'current_snake' passed to minimax
    # should already be a valid state before exploring further.

    # Check if the current state is already game over (e.g., if this state was reached after a move that caused a collision)
    game_over_at_current_state, _ = check_collisions(current_snake, current_food, BOARD_WIDTH, BOARD_HEIGHT)
    if game_over_at_current_state:
        return -10000000 + current_score # A very low score, with slight preference for higher scores before death

    # Base Case 2: Max depth reached, evaluate the state
    if depth == max_depth:
        return evaluate_state(current_snake, current_food, current_score, BOARD_WIDTH, BOARD_HEIGHT)

    # Recursive Step: Explore possible moves
    # Use get_possible_moves_corrected for better self-collision detection
    possible_moves = get_possible_moves_corrected(current_snake, BOARD_WIDTH, BOARD_HEIGHT)

    # If there are no possible moves, it's a losing state (stuck)
    if not possible_moves:
        return -10000000 + current_score

    best_score = -float('inf')

    for move in possible_moves:
        # Simulate the next state using update_game_state
        # Note: update_game_state returns new_snake, new_food, new_score, game_over
        # We need to explicitly check if this simulated move leads to game_over,
        # but update_game_state already incorporates check_collisions.

        # Create deep copies for simulation to avoid modifying original lists
        sim_snake = list(current_snake)
        sim_food = current_food # Tuples are immutable
        sim_score = current_score

        # update_game_state modifies the snake list. We need a temporary copy.
        # Let's create a temporary snake to pass to update_game_state that reflects the current state
        # for a specific move. `move_snake` is the first step of `update_game_state`.

        # First, apply the move to the snake to get the head position. This is the new_snake *before* tail pop
        next_head = get_next_head_position(sim_snake[0], move)
        temp_snake_after_move = [next_head] + sim_snake # Snake with new head, before food check/tail pop

        # Now, check for food collision at this simulated step
        game_over_after_move, food_eaten_after_move = check_collisions(temp_snake_after_move, sim_food, BOARD_WIDTH, BOARD_HEIGHT)

        if game_over_after_move:
            # If this specific move leads to game over, assign a low score
            score = -10000000 + sim_score
        else:
            # If not game over, update the state based on food eaten
            if food_eaten_after_move:
                next_snake_state = temp_snake_after_move # Snake grows
                next_score_state = sim_score + 1
                next_food_state = generate_food(next_snake_state) # New food location
            else:
                next_snake_state = temp_snake_after_move[:-1] # Remove tail
                next_score_state = sim_score
                next_food_state = sim_food # Food remains

            # Recursively call minimax for the next state
            score = minimax(next_snake_state, next_food_state, next_score_state, move, depth + 1, max_depth)

        best_score = max(best_score, score)

    return best_score

# Test the minimax function (conceptual test, as it requires a full game state simulation)
print("Minimax function defined. Testing will be done in the next step when ai_get_move is implemented.")

# Example of how it might be called:
# best_score_for_right = minimax(snake_initial_test, food_initial_test, score_initial_test, 'RIGHT', 0, 3)
# print(f"Max score for moving RIGHT: {best_score_for_right}")

Minimax function defined. Testing will be done in the next step when ai_get_move is implemented.


In [11]:
def ai_get_move(current_snake, current_food, current_score, current_direction, max_depth=3): # Added a default max_depth for convenience
    possible_moves = get_possible_moves_corrected(current_snake, BOARD_WIDTH, BOARD_HEIGHT)

    if not possible_moves:
        return current_direction # If no valid moves, maintain current direction (will lead to game over)

    best_move = None
    best_score = -float('inf')

    # Iterate through each possible move and call minimax to evaluate its long-term score
    for move in possible_moves:
        # Simulate the initial move for minimax to evaluate
        next_head = get_next_head_position(current_snake[0], move)
        temp_snake_after_move = [next_head] + list(current_snake) # Create a copy to avoid modifying original

        game_over_after_move, food_eaten_after_move = check_collisions(temp_snake_after_move, current_food, BOARD_WIDTH, BOARD_HEIGHT)

        if game_over_after_move:
            # This move leads to immediate death, assign a very low score
            score = -10000000 + current_score
        else:
            # If not game over, determine the next state
            if food_eaten_after_move:
                next_snake_state = temp_snake_after_move # Snake grows
                next_score_state = current_score + 1
                next_food_state = generate_food(next_snake_state) # New food location
            else:
                next_snake_state = temp_snake_after_move[:-1] # Remove tail
                next_score_state = current_score
                next_food_state = current_food # Food remains

            # Call minimax from depth 1 as we've already made the first move
            score = minimax(next_snake_state, next_food_state, next_score_state, move, 1, max_depth)

        if score > best_score:
            best_score = score
            best_move = move

    return best_move

# Test the ai_get_move function
# Reset global state for a clean test
snake = [(10, 10), (9, 10), (8, 10)]
food = generate_food(snake) # Ensure food is generated based on initial snake
score = 0
direction = 'RIGHT'
BOARD_WIDTH = 20
BOARD_HEIGHT = 20

print(f"Initial State for AI: Snake={snake}, Food={food}, Score={score}, Direction={direction}")

# Let's see what move the AI suggests with a depth of 2
ai_suggested_move = ai_get_move(snake, food, score, direction, max_depth=2)
print(f"AI suggested move (max_depth=2): {ai_suggested_move}")

# Let's try with a specific scenario where the AI should prioritize food
snake_ai_test = [(5, 5), (4, 5)]
food_ai_test = (6, 5) # Food is to the right
score_ai_test = 0
direction_ai_test = 'RIGHT'

print(f"\nScenario for AI: Snake={snake_ai_test}, Food={food_ai_test}, Score={score_ai_test}, Direction={direction_ai_test}")
ai_suggested_move_food = ai_get_move(snake_ai_test, food_ai_test, score_ai_test, direction_ai_test, max_depth=1)
print(f"AI suggested move (to food, max_depth=1): {ai_suggested_move_food}")

# Scenario where AI might avoid a wall (if the current direction is towards a wall, it should change)
snake_near_wall_ai = [(18, 10), (17, 10), (16, 10)] # Head at (18,10), moving right will hit wall
food_near_wall_ai = (5,5)
score_near_wall_ai = 0
direction_near_wall_ai = 'RIGHT'

print(f"\nScenario for AI (near wall): Snake={snake_near_wall_ai}, Food={food_near_wall_ai}, Score={score_near_wall_ai}, Direction={direction_near_wall_ai}")
ai_suggested_move_wall = ai_get_move(snake_near_wall_ai, food_near_wall_ai, score_near_wall_ai, direction_near_wall_ai, max_depth=2)
print(f"AI suggested move (near wall, max_depth=2): {ai_suggested_move_wall}")

Initial State for AI: Snake=[(10, 10), (9, 10), (8, 10)], Food=(9, 0), Score=0, Direction=RIGHT
AI suggested move (max_depth=2): UP

Scenario for AI: Snake=[(5, 5), (4, 5)], Food=(6, 5), Score=0, Direction=RIGHT
AI suggested move (to food, max_depth=1): RIGHT

Scenario for AI (near wall): Snake=[(18, 10), (17, 10), (16, 10)], Food=(5, 5), Score=0, Direction=RIGHT
AI suggested move (near wall, max_depth=2): UP


In [12]:
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
pygame.init()

# 2. Initialize Pygame and set up the display window.
SCREEN_WIDTH = BOARD_WIDTH * CELL_SIZE
SCREEN_HEIGHT = BOARD_HEIGHT * CELL_SIZE

display = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Snake Game AI")

# 3. Define RGB color tuples
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREY = (50, 50, 50)

# 4. Create a function draw_snake(display, snake_body)
def draw_snake(display_surface, snake_body):
    for segment in snake_body:
        pygame.draw.rect(display_surface, GREEN, (segment[0] * CELL_SIZE, segment[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# 5. Create a function draw_food(display, food_position)
def draw_food(display_surface, food_position):
    if food_position:
        pygame.draw.rect(display_surface, RED, (food_position[0] * CELL_SIZE, food_position[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# 6. Create a function draw_grid(display) to optionally draw grid lines
def draw_grid(display_surface):
    for x in range(0, SCREEN_WIDTH, CELL_SIZE):
        pygame.draw.line(display_surface, GREY, (x, 0), (x, SCREEN_HEIGHT))
    for y in range(0, SCREEN_HEIGHT, CELL_SIZE):
        pygame.draw.line(display_surface, GREY, (0, y), (SCREEN_WIDTH, y))

def display_score(display_surface, score):
    font = pygame.font.Font(None, 36)
    text = font.render(f"Score: {score}", True, BLUE)
    display_surface.blit(text, (5, 5))

# 7. Modify the main_game_loop function
def main_game_loop_with_gui(run_ai=True):
    global snake, food, direction, score, BOARD_WIDTH, BOARD_HEIGHT

    game_over = False
    game_tick_speed = 150 # milliseconds per game tick (e.g., 150ms)

    # Re-initialize for a clean game start
    snake = [(10, 10), (9, 10), (8, 10)]
    direction = 'RIGHT'
    score = 0
    food = generate_food(snake)

    clock = pygame.time.Clock()

    print("\n--- Starting Game Loop with GUI ---")
    print(f"Initial State: Snake={snake}, Food={food}, Score={score}, Direction={direction}")

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True

            # For player control (optional, if AI is not running)
            if not run_ai and event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and direction != 'DOWN':
                    direction = 'UP'
                elif event.key == pygame.K_DOWN and direction != 'UP':
                    direction = 'DOWN'
                elif event.key == pygame.K_LEFT and direction != 'RIGHT':
                    direction = 'LEFT'
                elif event.key == pygame.K_RIGHT and direction != 'LEFT':
                    direction = 'RIGHT'

        if run_ai:
            # d. Call ai_get_move to get the AI's next move and update the direction
            ai_direction = ai_get_move(snake, food, score, direction, max_depth=3) # Use a reasonable depth
            if ai_direction: # Ensure AI returned a valid move
                direction = ai_direction

        # Update game state
        snake, food, score, game_over = update_game_state(
            snake, direction, food, score, BOARD_WIDTH, BOARD_HEIGHT
        )

        # c. Fill the background
        display.fill(BLACK)

        # e. Call draw_grid, draw_snake, draw_food, and display score
        draw_grid(display)
        draw_snake(display, snake)
        draw_food(display, food)
        display_score(display, score)

        # f. Update the entire screen
        pygame.display.flip()

        # h. Add a delay using the clock
        clock.tick(1000 // (game_tick_speed // 10)) # Control FPS based on game_tick_speed

        # Debug print for game state progression
        # print(f"Snake: {snake}, Food: {food}, Score: {score}, Game Over: {game_over}, Direction: {direction}")

    print("Game Over! Final Score:", score)
    print("--- Game Loop with GUI Ended ---")
    pygame.quit()

# Run the main game loop with GUI and AI
main_game_loop_with_gui(run_ai=True)


--- Starting Game Loop with GUI ---
Initial State: Snake=[(10, 10), (9, 10), (8, 10)], Food=(14, 3), Score=0, Direction=RIGHT
Game Over! Final Score: 32
--- Game Loop with GUI Ended ---


## Final Task

### Subtask:
Demonstrate the functional Snake Game AI and summarize the learning outcomes and potential for further exploration.
